Thankfully [tafsir_api](https://github.com/spa5k/tafsir_api) has many tafsirs sectioned per Ayah. The 114 for Waseet have been downloaded and will be merged here:

In [1]:
import glob
import json

# Get list of JSON file paths sorted by filename
jsons = sorted(glob.glob('Tafsirs/*.json'), key=lambda x: int(x.split("/")[-1].split(".")[0]))

full_tafsir = []
for fname in jsons:
    with open(fname) as infile:
        tafsir = json.load(infile)
        tafsir = sorted(tafsir["ayahs"], key=lambda x: x["ayah"])
        full_tafsir.append(tafsir)  

# Save the merged JSON to a file
with open('tafsirs.json', 'w') as f:
    json.dump(full_tafsir, f, ensure_ascii=False, indent=4)


Now let's read the tafsir json and make some transformations for readability.

Remove extra characters (e.g., dots) between braces so no new lines are inserted there later:

In [2]:
def remove_chars_between_braces(text):
    inside_braces = False
    result = ''
    for char in text:
        if char == '{':
            inside_braces = True
        elif char == '}':
            inside_braces = False
        elif inside_braces and (char in ['.', ".", "_"]) or (char.isspace() and char !=" "):
            continue
        result += char
    return result

# Example usage:
arabic_string = "مرحبا. هذا هو النص العربي {مع. النقاط. ولكن. لا نريدها. هنا    }."
cleaned_string = remove_chars_between_braces(arabic_string)
print(cleaned_string)

مرحبا. هذا هو النص العربي {مع النقاط ولكن لا نريدها هنا    }.


Make the 12 (as in the if conditions) words before the colon italic and bold if they are at the start of the sentence. What italic will do is change the color (has a custom style in the code) but not make actual text italic

In [3]:
import re

def make_before_colons_italic(input_string):
    lines = input_string.split('<br><br>')
    processed_lines = []

    for line in lines:
        parts = line.split(':')
        if len(parts) > 1:
            before_colon = parts[0]
            words_before_colon = len(before_colon.split())
            if words_before_colon <= 12:
                line = '<b><i>' + before_colon.strip() + ':</i></b>' + ':'.join(parts[1:])
        processed_lines.append(line)

    return '<br><br>'.join(processed_lines)

Extra `<br><br>` tags (white space) will appear at the end of the tafsir after processing so this fixes it

In [4]:
def remove_final_br_tags(string):
    if string.endswith('<br><br>'):
        return string[:-8]  # Removing the last 8 characters ("<br><br>")
    else:
        return string

Can remove tashkeel for stemming (not really used here)

In [5]:
def remove_tashkeel(text):
    return re.sub(u'[\u064e\u064f\u0650\u0651\u0652\u064c\u064b\u064d\u0640\ufc62]','',text)

Special words that should be colored by wrapping in an italic tag

In [6]:
special_list = [
"سبحانه",
"المراد",
"الله",
"تعالى",
"سبحانه",
"صلى",
"قال",
"قوله",
"عليه",
"الآية",
"ابن",
"القرآن",
"والمراد",
"الرسول",
"وقوله",
"فقال",
"الكريم",
"موسى",
"النبى",
"وسلم",
"القيامة",
"الإِمام",
"بالله",
"سيدنا",
"محمد",
"آله",
"وعلى",
"وصحبه",
"وسلم",
"وصلى",
]

In [10]:
import json
import re
from tqdm.notebook import tqdm

# read tafsirs.json
with open('tafsirs.json') as f:
    tafsirs = json.load(f)
    
with open ('suras.json') as f:
    suras = json.load(f) 

# Iterate through each list with tqdm
for i, surah in tqdm(enumerate(tafsirs), total=len(tafsirs)):
    for j, tafsir in enumerate(surah):
        tafsirWords = tafsir['text'].split(" ")
        
        # Any word just before : should be italic (i.e., be colored as we mentioned above)
        for k, tafsirWord in enumerate(tafsirWords[:-1]):
            if tafsirWords[k+1] == ":" :
                tafsir['text'] = re.sub(r'\b{}\b'.format(re.escape(tafsirWords[k])), f'<i>{tafsirWords[k]}</i>', tafsir['text'])
        
        # special words should be italic (i.e., be colored as we mentioned above)
        for word in special_list:
            pattern = f'(?<![^\W\d_])({word})(?![^\W\d_])'  # Match the word only if not preceded or followed by a non-alphanumeric character
            tafsir['text'] = re.sub(pattern, r'<i>\1</i>', tafsir['text'])


        # if you ever find {...} surround it with <b></b> to make it bold
        tafsir['text'] = tafsir['text'].replace('{', '{ <b>')
        tafsir['text'] = tafsir['text'].replace('}', '</b> }')
        tafsir['text'] = tafsir['text'].replace('(', '{ <b>')
        tafsir['text'] = tafsir['text'].replace(')', '</b> }')
        tafsir['text'] =  tafsir['text'].replace('<b> ', '<b>')
        tafsir['text'] =  tafsir['text'].replace(' </b>', '</b>')

        # remove any special characters within {.*} for purpose mentioned above
        tafsir['text'] = remove_chars_between_braces(tafsir['text'])

        # Add new lines after '. ' or '...' unless it's followed by a quote
        pattern = r'\.(?!")(?! ")(?!\.)(?! \.)'
        tafsir['text'] = re.sub(pattern, '.<br><br>', tafsir['text'])
        pattern = r'\.\.\.(?!")(?!\.)'
        tafsir['text'] = re.sub(pattern, '...<br><br>', tafsir['text'])

        # Remove spaces preceding a dot in the 'text' field
        tafsir['text'] = tafsir['text'].replace(' .', '.')
        
        # Remove floating dots
        tafsir['text'] = tafsir['text'].replace('<br>.<br>', '')

        # Make words before colon colored
        tafsir['text'] = make_before_colons_italic(tafsir['text'])

        # Remove extra new lines at the end
        tafsir['text'] =  remove_final_br_tags(tafsir['text'])
        
# save as surahTafsir
with open('surahTafsirs.json', 'w') as f:
    json.dump(tafsirs, f, ensure_ascii=False, indent=4) 

  0%|          | 0/114 [00:00<?, ?it/s]

Test parsing and reformatting:

In [13]:
from IPython.display import display_html

display_html(
    f"""<div style="direction:rtl">
    {tafsirs[17][19]['text']}
    </div>
    """
    , raw=True)

وقوله - سبحانه -: { وكذلك بَعَثْنَاهُمْ لِيَتَسَآءَلُوا بَيْنَهُمْ } ، بيان للعلة التى من أجلها بعث أصحاب الكهف من نومهم الطويل. أى : وكما أنمناهم تلك المدة الطويلة ، بعثناهم من نومهم بعدها ، ليسأل بعضهم بعضا ، وكأنهم قد أحسوا بأن نومهم قد طال. والاقتصار على التساؤل الذى حصل الإِيقاظ من أجله ، لا ينفى أن يكون هناك أسباب أخرى غيره حصل من أجلها إيقاظهم ، وإنما أفرده - سبحانه - بالذكر لاستتباعه لسائر الآثار الأخرى. ثم حكى - سبحانه - بعض تساؤلهم فقال : { قَالَ قَائِلٌ مِّنْهُمْ كَم لَبِثْتُمْ } أى : كم مكثتم مستغرقين فى النوم فى هذا الكهف. فأجابه بعضهم بقوله : { لبثنا يوماً } لظنهم أن الشمس قد غربت ، فلما رأوها لم تغرب بعد قالوا : { أَوْ بَعْضَ يَوْمٍ } أى : مكثنا نائمين بعض ساعات اليوم. ويصح أن تكون أو للشك. أى قال بعضهم فى الرد على سؤال السائل كم لبثتم ، لبثنا فى النوم يوما أو بعض يوم ، لأننا لا ندرى على الحقيقة كم مكثنا نائمين. ثم حكى القرآن أن بعضهم رد عِلْمَ مقدار مدة نومهم على جهة اليقين إلى الله - تعالى - فقال : { قَالُواْ رَبُّكُمْ أَعْلَمُ بِمَا لَبِثْتُمْ } أى : ربكم وحده هو العليم بمقدار الزمن الذى قضيتموه نائمين فى هذا الكهف. قال الآلوسى : وهذا رد منهم على الأولين ، على أحسن ما يكون من مراعاة حسن الأدب ، وبه كما قيل يتحقق التحزب إلى الحزبين المعهودين فيما سبق فى قوله - تعالى - { لِنَعْلَمَ أَيُّ الحِزْبَيْنِ } وقال بعضهم : وقد استدل ابن عباس على أن عدد الفتية سبعة بهذه الآية ، لأنه قد قال فى الآية : قال قائل منهم ، وهذا واحد ، وقالوا فى جوابه : لبثنا يوما ، أو بعض يوم وهو جمع وأقله ثلاثة ، ثم قالوا : ربكم أعلم بما لبثتم ، وهذا قول جمع آخرين فصاروا سبعة. ثم بين - سبحانه - ما قالوه بعد أن تركوا الحديث فى مسألة الزمن الذى قضوه نائمين فى الكهف فقال - تعالى - : { فابعثوا أَحَدَكُمْ بِوَرِقِكُمْ هذه إلى المدينة فَلْيَنْظُرْ أَيُّهَآ أزكى طَعَاماً فَلْيَأْتِكُمْ بِرِزْقٍ مِّنْهُ وَلْيَتَلَطَّفْ وَلاَ يُشْعِرَنَّ بِكُمْ أَحَداً }. أى : كفوا عن الحديث فى مسألة المدة التى نمتموها ، فعلمها عند الله ، وابعثوا أحدكم { بورقكم }. أى : بدراهمكم المضروبة من الفضة ، { إلى المدينة } التى يوجد بها الطعام الذى نحن فى حاجة إليه ، والتى هى أقرب مكان إلى الكهف. قالوا : والمراد بها مدينتهم التى كانوا يسكنونها قبل أن يلجأوا إلى الكهف فراراً بدينهم. { فَلْيَنْظُرْ أَيُّهَآ أزكى طَعَاماً } أى : ومتى وصل إلى المدينة ، فليتفقد أسواقها ، وليتخير أى أطعمتها أحل وأطهر وأجود وأكثر بركة. { فَلْيَأْتِكُمْ بِرِزْقٍ مِّنْهُ وَلْيَتَلَطَّفْ } أى : فليأتكم بما يسد جوعكم من ذلك الأزكى طعاما ، فيكون الضمير فى " منه " للطعام الأزكى.

In [88]:
tafsirs[14][0]['text']

'سورة الحجر من السور التى افتتحت ببعض حروف التهجى { <b>الر</b> }.<br><br>وقد بينا - بشئ من التفصيل - عند تفسيرنا <i>لسورة</i> : البقرة ، وآل عمران ، والأعراف...<br><br>آراء العلماء فى هذه الحروف التى افتتحت بها بعض سور القرآن الكريم.<br><br>وقلنا ما <i>خلاصته</i> : من العلماء من يرى أن المعنى المقصود منها غير معروف لأنها من المتشابه الذى استأثر الله بعلمه...<br><br>ومنهم من يرى أن المعنى المقصود منها معلوم ، وأنها ليست من المتشابه ، بل هى أسماء للسور التى افتتحت بها...<br><br> أو هى حروف مقطعة بعضها من أسماء الله ، وبعضها من صفاته...<br><br>ثم <i>قلنا</i> : ولعل أقرب الآراء إلى الصواب أن <i><i>يقال</i></i> : إن هذه الحروف المقطعة ، قد وردت فى افتتاح بعض السور؛ للإِشعار بأن هذا القرآن الذى تحدى الله به المشركين ، هو من جنس الكلام المركب من هذه الحروف التى يعرفونها ، ويقدرون على تأليف الكلام منها ، فإذا عجزوا عن الإِتيان بسورة من مثله ، فذلك لبلوغه فى الفصاحة والحكمة مرتبة يقف فصحاؤهم وبلغاؤهم دونها بمراحل.<br><br>وفضلاً عن ذلك فإن تصدير بعض السور بمثل هذه الحروف المقطعة ، يجذب أنظار الم

Get most frequest words to then add in `special_words` list above.

In [85]:
from collections import Counter
from tqdm import tqdm

# Concatenate all text
all_text = ' '.join([tafsir['text'] for surah in tafsirs for tafsir in surah])

# Tokenize the text into words
words = all_text.split()

# Count the frequency of each word
word_counts = Counter(words)

# Sort the words based on their frequencies
sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

# Select the top 200 words
top_200_words = sorted_words[:200]

# Print the top 200 words
for i, (word, freq) in tqdm(enumerate(top_200_words), total=200):
    print(f"{i+1}. {word}")

100%|██████████| 200/200 [00:00<00:00, 279433.98it/s]

1. ،
2. -
3. :
4. من
5. فى
6. الله
7. على
8. أن
9. ما
10. تعالى
11. إلى
12. عليه
13. _أى_
14. عن
15. لا
16. "
17. _سبحانه_
18. هذا
19. في
20. أو
21. ذلك
22. صلى
23. ثم
24. _قال_
25. الذى
26. به
27. هذه
28. هو
29. وسلم
30. الذين
31. ولا
32. كان
33. التى
34. وهو
35. كل
36. بعد
37. قال
38. كما
39. _فقال_
40. إذا
41. قد
42. بين
43. قوله
44. لأن
45. _قوله_
46. أنه
47. إلا
48. له
49. فيه
50. لم
51. وقوله
52. لهم
53. وقد
54. إن
55. ومن
56. الآية
57. الناس
58. ابن
59. مع
60. ".
61. الكريمة
62. القرآن
63. يكون
64. حتى
65. بما
66. عند
67. وما
68. عليهم
69. بأن
70. فإن
71. بها
72. رسول
73. لما
74. وفى
75. هؤلاء
76. السلام
77. _والمراد_
78. يا
79. الحق
80. بن
81. _الله_
82. فِي
83. يوم
84. إليه
85. فيها
86. **}
87. المؤمنين
88. بعض
89. بمعنى
90. غير
91. الرسول
92. _تعالى_
93. منه
94. أيها
95. الأرض
96. لأنه
97. _وقوله_
98. _أي_
99. الدنيا
100. الذي
101. فقد
102. منهم
103. فقال
104. بل
105. أهل
106. عنه
107. وأن
108. أى_
109. مِن
110. أنهم
111. وقال
112. كانوا
113. أى
114. بيان
115. و
116. الكريم
1